<a href="https://colab.research.google.com/github/AnaJuvencio/Aprendizado_Profundo_Reconhecimento_Visual/blob/main/Projeto_Aprendizado_Profundo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação de Bibliotecas

In [1]:
!pip install datasets tensorflow matplotlib scikit-learn
import tensorflow as tf

Carregando o dataset TrashNet

In [2]:
from datasets import load_dataset

ds = load_dataset("garythung/trashnet")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

dataset-original.zip:   0%|          | 0.00/3.63G [00:00<?, ?B/s]

dataset-resized.zip:   0%|          | 0.00/42.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5054 [00:00<?, ? examples/s]

Conferir chave disponíveis

In [ ]:
print(ds)
print(ds['train'][0])